# Sequence to Sequence models and NMT

Повторение рекуррентных нейронных сетей 

Повтороение нейронных рекурректных архитектур



Что такое sequence-to-sequence?

Sequence-to-sequence learning (Seq2Seq) - это обучение моделей, для преобразования последовательностей из одной предметной области (например, предложения на английском языке) в последовательность другой предметной области (например, предложения на французском языке).

**"the cat sat on the mat" -> [Seq2Seq model] -> "le chat etait assis sur le tapis"**

Seq2Seq модели активно используются в машинном переводе или для вопросно ответных систем, но не ограничиваются этим.

Есть несколько способов работы с этой задачей, или используя RNN, или используя 1D convnets. На этом занятии мы сфокусируемся на RNN.


Простейший случай: входная и выходная последовательности одной и той же длины.

Когда входная и выходная последовательности имеют одну и ту же длину, реализовать такие модели нет никаких проблем: использовать простой слой LSTM/GRU (или их стэк).

Сложение чисел, представленных как символы:
![](https://blog.keras.io/img/seq2seq/addition-rnn.png)

Код для этого примера можно найти здесь: https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py 

Одним из недостатков этого подхода является то, что он предполагает, что возможно сгенерировать 

    target[...t]

по заданному

    input[...t].

В некоторых случаях это работает (например, сложение чисел, представленных как строки), но это не работает для более общих случаев.

В общем случае, информация о всей взодной последовательности необходимо для начала генерации целевой последовательности.

Общий случай: канонический sequence-to-sequence

В общем случае входные последовательности и выходные последовательности имеют разную длину (например, машинный перевод), и вся входная последовательность требуется для начала прогнозирования цели. Для этого требуется более точная настройка, на которую обычно и ссылаются люди, когда упоминают «sequence to sequence models» без какого-либо дальнейшего контекста. Вот как это работает:

* Слой RNN (или их стек) выступает как кодировщик (encoder): он обрабатывает входную последовательность и возвращает своё внутреннее состояние. Обратите внимание, что мы отбрасываем выходы кодировщика (encoder) RNN, **восстанавливая только состояние**. Этот состояние выступает как "контекст" для декодера (decoder) на сл. шаге.
* Другой слой RNN (или их стек) выступает в качестве "декодера" (decoder): он обучается предсказывать следующие символы целевой последовательности при заданных предыдущих символах целевой последовательности. В частности, он обучается преобразовывать целевые последовательности (target sequences) в те же последовательности, но смещенные на один timestep, процесс обучения называется "teacher forcing". Importantly, the encoder uses as initial state the state vectors from the encoder, which is how the decoder obtains information about what it is supposed to generate. 
Фактически, декодер обучается генерировать

    target[t+1...]
при заданных

    target[...t]
при условии входной последовательности.

![](https://blog.keras.io/img/seq2seq/seq2seq-teacher-forcing.png)


В режиме inference, т.е. когда мы хотим декодировать неизвестную входную последовательность, мы проходим через несколько измененный процесс:
* Закодировать входную последовательность в векторы состояний;
* Начать с целевой последовательности (target sequence) ращме 1 (символ ситарта последовательности)
* Передать векторы состояний и 1 символ целевой последовательности (1-char target sequence) декодеру для порождения предсказания сл. символа
* Выбрать следующий символ на базе этих предсказаний (например, используя argmax);
* Добавить этот символ к целевой последовательности (target sequence)
* Повторять пока мы не сгенерируем символ конца последовательности.

![](https://blog.keras.io/img/seq2seq/seq2seq-inference.png)

Тот же процесс может быть также использовать для обучения без "teacher forcing", т.е. реинжекцией предсказаний декодера в декодер.


Encoder конвертирует последовательность атрибутов в вектор большой размерности (как правило, обозначается HIDDEN_SIZE). В то время как конечный результат является последовательностью атрибутов (символов), каждый символ предсказывается генеративной RNN (декодером, decoder).

Итак, вектор, генерируемый из кодировщика (encoder) нуждается в "умножении" на длину выходной последовательности. Он становится входом в декодирующую RNN.

Декодирующая RNN возвращает последовательности. Размерность выхода (timesteps, hidden_size). Поскольку каждая сеть требует в конце полносвязный слой, после декодирующей RNN следует TimeDistributedDense слой, который выдает класс символа.

TimeDistributedDense - это еще одно имя для нескольких плотных слоёв. Просто происходит применение Dense слоя на длину задаваемую timestep. За этим слоем следует обычный слой ктивации для генерации вероятностей.


## Sequence to sequence : Keras Example

Иллюстрируем в коде.

Для нашей реализации будем использовать пары переводов предложений на английском и французском языках.

* Наборы переводов: http://www.manythings.org/anki/
* Датасет: http://www.manythings.org/anki/fra-eng.zip

Реализуем **character-level sequence-to-sequence model**, обрабатывающая входную последовательность посимвольно, и генерирующую выходную последовательность посимвольно. 

Краткое описание процесса:

1. Преобразовать предложения в 3 массива NumPy: 
    * encoder_input_data - трехмерный массив размерности (num_pairs, max_engslish_sentence_length, num_english_characters), содержащий one-hot представление английских предложений.
    * decoder_input_data - трехмерный массив размерности (num_pairs, max_french_sentence_length, num_french_characters), содержащий one-hot представление французских предложений;
    * decoder_target_data - то же самое, что и decoder_input_data, но со смещением на один timestep. То есть decoder_target_data[:, t, :] станет тем же самым, что и decoder_input_data[:, t+1, :].
    
* Обучить базовую LSTM Seq2Seq модлель предсказывать decoder)target_data при заданных encoder_input_data и decoder_input_data. Модель использует teacher forcing.

* Декодирование некоторых предложения для проверки работоспособности модели.

Поскольку процесс обучения и процесс вывода (inference process, decoding sentences) сильно отличаются, будем использовать различные модели, хотя все они используют одни и те же внутренние слои.

#### Training model

Модель обучения использует 3 ключевые особенности RNN в Keras:
* аргумент *return_state*, задаваемый в конструктере RNN слоя конфигурирует RNN возвращать список, где первая сущность является выходом и следующие сущности - внутренние состояния RNN. Используется для восстановления состояний кодировщика (encoder).
* аргумент initial_state указывает начальное состояние(я) RNN. Используется для передачи состояний кодировщика, как начальные состояния декодера.
* аргумент return_sequences, задаваемый в конструкторе, конфигурирует RNN возвращать полную последовательность выходов (вместо последнего выхода, по умолчанию). Используется в декодере.


In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Модель обучается достаточно просто: в две строки

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

#### Inference model

1. Закодируем входное предложение и получим начальное состояние декодера
* Запустим один шаг декодера с начальным состоянием и в качестве target - токен "start of sequence". Выходом будет следующий символ целевого предложения.
* Добавить прпедсказанный целевой символ и повторить процесс.

Код, соответствующий процессу порождения:



In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

Используем его в цикле при выводе:

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

Полный код приведен по ссылке:
https://github.com/fchollet/keras/blob/master/examples/lstm_seq2seq.py



## Neural Machine Translation

Одной из самых ранних целей компьютерной лингвистики был автоматический перевод текстов с одного языка на другой. Автоматический машинный перевод, возможно, одна из самых сложных задач ИИ, связанная с естественны языком.

Классические системы, основанные на правилах, которые использовались ля этой задачи стали замещаться статистическими методами в 90-х годах.

Позднее, глубокие нейронные сети достигли современных результатов в области машинного перевода и эффективности моделей машинного перевода.

### Что такое машинный перевод (machine translation)?

Машинный перевод - задача автоматического преобразования исходного текста на одном языке в текст на другом языке.

Если задана текстовая последовательность на исходном языке (source language), нет одного самого лучшего перевода этого текста на другой язык. Это обусловлено неоднозначностью и гибкостью человеческого языка.

И это делает задачу автоматического машинного перевода по-настоящему трудной, возможно одной из самых сложных задач в ИИ:

<hr>
<i>
    The fact is that accurate translation requires background knowledge in order to resolve ambiguity and establish the content of the sentence.</i>

**Artifical Intelligence, A Modern Approach**
<hr>

Классические методы машинного перевода часто включают в себя правила для преобразования текста на исходном языке в текст на целевом языке. Правила зачастую разрабатываются лингвистами и щатрагивают лексический, синтаксический и семантический уровни. 

Этот подход к машинному переводу называется Rule-based Machine Translation (RBMT). 
    
<hr>
<i>
RBMT is characterized with the explicit use and manual creation of linguistically informed rules and representations.</i>

**Handbook of Natural Language Processing and Machine Translation**
<hr>
    
Ключевые ограничения классических подходов к машинному переводу заключаются в том, что они требуют как целой армии экспертов для разработки правил, и большого числа правил и сопутствующих исключений.

### Что такое статистический машинный перевод (statistical machine translation)?

SMT использует статистические модели, чтобы научиться переводить текст из исходного языка на целевой язык по заданному объемному параллельному корпусу.

Задача использования статистической модели может быть формально поставлена сл. образом:
        
        
Эта формальная спецификация производит максимизацию выходной последовательности при заданной входной последовательности текста. Она также означает, что существует набор кандидатов, а также необходимость поиска (декодирования) для выбора одного наиболее вероятного перевода из выходного вероятностного распределения модели.

<hr>
<i>Задан текст на исходном языке, какой наиболее вероятный перевод в целевом языке? [...] как нам следует сконструировать статистическую модель, которая назначает высокие вероятности "хорошим" переводам, и низкие вероятности "плохим" переводам?
</i>

<b>Syntax-based Statistical Machine Translation</b>
<hr>
        
Этот подход data-driven, т.е. требует только большой корпус примеров с исходными и целевыми предложениями. Это означает, что больше не нужно задавать правила перевода.

<hr>
<i>Этот подход не нуждается в сложной онтологии или интерлингвальных концепциях, и не требует конструирования грамматик целевых и исходных языков, а также размеченного корпуса синтаксических разметок. Всё что необходимо, это т.н. data-sample translations, на базе которых модель перевода может быть обучена.
</i> 

**Artifical Intelligence, A Modern Approach**
<hr>

Наиболее широко используемыми техниками были т.н. phrase-based подходы, которые концентрировались на переводе подпоследовательностей фрагментов исходного текста.

<hr>
<i>
        SMT был доминирующей парадигмой перевода десятилетиями. Практические реализации SMT были преимущественно phrase-based системами (PBMT) , в которых переводились последовательности слов или фразы, длина которых могла быть различной.
</i>

**Google's Neural Machine Translation System: Bringing the Gap between Human and Machine Translation.**
<hr>

Несмотря на свою эффективность, методы статистического машинного перевода страдали от сложных фраз. Сильная концентрация на data-driven подходах также означает, что методы могут игнорировать важные синтаксические различия, известные лингвистам. Наконец, статистические подходы требуют тщательной настройки каждого модуля в translation pipeline.



### Что такое нейронный машинный перевод (neural machine translation)?

Нейронный машинный перевод (NMT) использует модели нейронных сетей для обучения статистическим моделям машинного перевода. Ключевым преимуществом является единая система, которая может обучаться на исходных и целевых текстах, и не требует конвеера свециализированных систем, используемых в SMT.

<br />
        В отличии от традиционных phrase-based translation system которые состоят из множества небольших подкомпонентов, настраиваемых независимо друг от друга, нейронные машинный перевод пытается построить и обучить единую, достаточно большую нейронную сеть, которая читает предложения и выводит корректный перевод.
<br />
<p align="center"><b>Neural Machine Translation by Jointly Learning to Align and Translate</b></p>
        
Таким образом, NMT системы, являются конечными системами из одной модели, требуемой для перевода.

<br/>
        Сильная сторона NMT состоит в способности обучаться непосредственно преобразуя входной текст в соответствующий выходной текст.
<br/>
**Google's Neural Machine Translation System: Bringing the Gap between Human and Machine Translation**

#### Encoder-Decoder Model
Модели многослойного перцептрона могут быть использованы для машинного перевода, хотя эти модели ораничены входными последовательностями **фиксированной длины**, где выход должен быть одной и той же длины.

Эти ранние модели позже были значительно улучшены посредством использования рекуррентных нейронных сетей, организованных в архитектуру кодировщика-декодировщика (encoder-decoder architecture), позволяющую использовать входные и выходные последовательности переменной длины.
<br />
*Кодировщик (encoder) нейронной сети считывает и кодирует исходное предложение в вектор фиксированной длины. Декодер (decoder) затем выводит перевод из вектора кодировщика. Вся система encoder-decoder, состоящая из кодировщика и декодера для языковой пары обучается совместно максимизировать вероятность корректного перевода по заданному исходному предложению.*
<br/>
    **Neural Machine Translation by Jointly Learning to Align and Translate**
    
Ключевой особенностью архитектуры кодировщика-декодера является способность модели кодировать исходный текст во внутреннюю репрезентацию фиксированной длины, называемую вектором контекста (context vector). Интересно также то, что контекстный вектор, будучи однажды закодированным, может использоваться разными декодирующими системами (decoding systems), для перевода контекста на разные языки.

<br />
    ... одна модель сначала считывает входную последовательность и порождает структуру данных, суммаризирующую входную последовательность. Мы назовем эту суммаризацию "контекстом" и будем обозначать C. [...] Вторая модель, обычна представляющая собой RNN затем считывает контекст C и порождает предложение на целевом языке.
<br />
    **Deep Learning**
    

#### Encoder-Decoders with Attention

Несмотря на свою эффективность, Encoder-Decoder архитектуры имеют проблемы с достаточно длинными последовательностями переводимых текстов. Корень этой проблемы во внутренней репрезентации фиксированной длины, которая используется при декодировании каждого слова в выходной последовательности. Решени состоит в использовании механизма внимания (attention mechanism), позволяющий модели обучаться, куда необходимо обращать внимание во входной последовательности при декодировании каждого слова выходной последовавательности.

<br />
Использование репрезентации фиксированного размера для того, чтобы захватить все семантические детали очень длинного предложения [..] чрезвычайно сложно. [..] Более эффективным подходом является чтение всего предложения или параграфа [..], затем породить перевод слова по одному, каждый раз фокусируясь на разных частях входного предложения для получения семантических деталей, требуемых для порождения следующего выходного слова.
<br/>
**Deep Learning**

Архитектура encoder-decoder с рекуррентной нейронной сетью со вниманием в настоящее время представляет собой state-of-the-art для ряда проблем в машинном переводе. Кроме того, эта архитектура является сердцем Google Neural Machine Translation system (GNMT), используемой в Google Translate.

<br/>
... текущие системы со вниманием представляют собой state-of-the-art в системах машинного перевода.
<br/>
**Neural Network Methods in Natural Language Processing**

Несмотря на чрезвычайную эффективность, системы нейронного машинного перевода продолжают иметь некоторые проблемы, такие как масштабирование на огромные словари слов и медленная скорость обучения этих моделей. 

<br/>
Три присущие нейронному машинному переводу слабые стороны [..]: долго обучение и скорость вывода, неэффективность при работе с редкими словами, а иногда и вовсе отказ переводить все слова в исходном предложении.
<br/>
**Google's Neural Machine Translation System: Bringing the Gap between Human and Machine Translation**

## Encoder-Decoder Models for NMT

Архитектура кодировщик-0декодер (encoder-decoder architecture for recurrent neural networks) является стандартным методом нейронного машинного перевода и в ряде случае превосходит классические методы статистического машинного перевода. Эта архитектура появилась в 2014 году, и очень быстро стала ключевой технологией в Google's translate service.


### Encoder-Decoder Architecture for NMT
Encoder-Decoder architecture стала чрезвычайно эффективным и стандартным подходом как для NMT так и для задач предсказания последовательности (sequence-to-sequence prediction).
Оснонвым достоинством подхода является возможность обучить одну модель для исходных и целевых предлложений и способность обрабатывать входные и выходные последовательности переменной длины. Свидетельством успеха этого метода является то, что эта архитектура стала ядром Google translation service.

<br />
Our model follows the common sequence-to-sequence learning framework with
attention. It has three components: an encoder network, a decoder network, and an attention network.
<br />
**Google’s Neural Machine Translation System: Bridging the Gap between Human and
Machine Translation**

Рассмотрим 2 разных исследовательских проекта, которые разработали одну и ту же архитектуру кодировщика-декодера в 2014 году.
* Sutskever NMT Model
* Cho NMT Model

### Sutskever NMT Model
Рассмотрим модель нейронного машинного перевода, разработанную Ilya Sutskever et al., описанную в 2014 году в статье "Sequence to Sequence Learning with Neural Networks". Будем называть её модель Sutskever NMT Model.

Это важная статья, поскольку именно в ней впервые введена модель Encoder-Decoder для машинного перевода и sequence-to-sequence learning., Это важнейшая модель в области машинного перевода и была одной из первых систем нейронного машинного перевода, которая превосходила baseline моделей статистического машинного перевода на больших задачах перевода.

#### Проблема
Модель применялась к проблеме перевода с английского на французский язык (WMT 2014 translation task). Задача перевода обрабатывала одно предложение в единицу времени и токен конца последовательности (end-of-sequence, <EOS>) добавляляс в конец выходной последовательности во время обучения, чтобы обозначить конец переведенного предложения.
Это позволяет модели быть способной прогнозировать выходную последовательность переменной длины. 

<br/>
Обратите внимание, что мы требуем, чтобы каждое предложение заканчивалось специальным символом конца предложения EOS, что позволяет модели определить распределение по последовательностям произвольной длины.
<br/>
**Sequence to Sequence Learning with Neural Networks**

Модель обучена на подмножестве из 12 миллионов предложений, в наборе данных состоящем из 348 миллионов французских слов и 304 миллионов английских слов.

Этот набор был выбран потому что он был предварительно обработан. Исходный словарь был сокращен до 160 000 наиболее часто встречающихся английских слов и 80 000 наиболее часто  встречающихся французских слов. Все неизвестные слова были заменены токеном UNK.

#### Model
Encoder-Decoder Architecture была разработана так, что входная последовательность (переменной дллины) полностью прочитывается и кодируется во внутреннюю репрезщентацию фиксированной длины. Сеть декодера затем использует эту внутреннюю репрезентацию для вывода слов, пока не будет достигнут токен конца последовательности. Как для кодировщика, так и для декодера были использованы LSTM сети.

<br/>
Идея использовать одну LSTM для чтения входной последовадовательности для получения достаточно большой векторной репрезентации фиксированной длины, а затем использовать другую LSTM сеть для извлечения выходной последовательности из этого вектора.
<br />
**Sequence to Sequence Learning with Neural Networks**


Конечная модель была ансамблем из 5 моделей глубокого обучения. 

Ниже изображена схема работы Encoder-Decoder Sutskever Model
![](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Depiction-of-Sutskever-Encoder-Decoder-Model-for-Text-Translation.png) 

#### Конфигурация модели

Конфигурация модели, описанная в статье Sequence to Sequence Learning with Neural Networks:

* Входные последовательности были "перевернуты" 
* Embedding layer размерности 1000 использовался для репрезентации входных слов;
* В качестве активации output layer'a использовался SoftMax;
* Вход и выход модели имели 4 слоя с 1 000 нейронов на слой;
* Модель обучилась за 7.5 эпох с затухающим learning rate;
* Размер batch'a 128 последовательностей;
* Использовались специальные подходы (gradient clipping) во время обучения дял минимизации риска взрыва градиента.
* Пакеты (batches) состояли из предложений примерно одинаковой длины.

Модель была обучена на машине с 8 GPU, где каждый слой запускался на различных GPU. Обучение заняло 10 дней.

<br />
Результирующая реализация достигла скорость 6 300 (английских и французских) слов в секунду с размером minibatch = 128. Обучение заняло около 10 дней
< br>
**Sequence to Sequence Learning with Neural Networks**

#### Результаты

Система достигла BLEU в 34.81, что является хорошим показателем, по сравнению с baseline от SMT в 33.30.

Важно, что это был первый пример системы нейронного машинного перевода, который превзошел phrase-based SMT baseline на масштабируемых проблемах.

<br/>
... we obtained a BLEU score of 34.81 [...] This is by far the best result achieved by
direct translation with large neural networks. For comparison, the BLEU score of
an SMT baseline on this dataset is 33.30
<br/>
**Sequence to Sequence Learning with Neural Networks**

Конечная модель использовала переоценку списка наилучших переводов и смогла улучшить BLEU до 36.5, что стало достаточно близко лучшему результату 37.0.

### Cho NMT Model

Рассмотрим модель описанную Kyunghyun Cho, et al. в их статье *Learning Phrase Representations using RNN EncoderDecoder for Statistical Machine Translation*. В дальнейшем будем ссылаться на жту модель, как **Cho NMT Model**. 

Важно, что Cho Model используется только для оценки кандидатов в переводы и напрямую не используется для перевода, в отличии от модели Sutskever'a. 
Хотя есть ряд расширений, позволяющих использовать эту модель напрямую.

#### Проблема
Как и прежде, проблема состоит в переводе с английского языка на французский (задача машинного перевода WMT 2014). Исходный и целевой словари были ограничены наиболее частовстречаемыми 15 000 французскими и английскими словами (что покрывает 93% набора данных) и слова не из словаря заменялись спецтокеном UNK.

<br />
.. called RNN Encoder-Decoder that consists of two recurrent neural networks
(RNN). One RNN encodes a sequence of symbols into a fixed-length vector representation, and the other decodes the representation into another sequence of symbols. 
<br />
**Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation**

![](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Depiction-of-the-Encoder-Decoder-architecture.png)

Реализация не использует LSTM, вместо этого применяется упрощенный тип: GRU (gated recurrent unit).

<br/>
... we also propose a new type of hidden unit that has been motivated by the LSTM unit but is much simpler to compute and implement.
<br/>
Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

#### Конфигурация

Конфигурация модели, описанная в статье *Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation*:
* Для репрезентации embedding'ов входных слов использовались вектора размерности 100;
* Кодировщик и декодировщик были сконфигнурированы с 1 слоем из 1000 GRU нейронов.
* 500 Maxout units pooling 2 input было использовано после декодирования
* Batch size = 64 предложения.

Модель обучалась примерно 2 дня.

#### Расширения

В статье "On the Properties of Neural Machine Translation: Encoder-Decoder Approaches" Cho, et al. исследователи ограничения своей модели. 

Они обнаружили, что производительность быстро снижается с ростом длины входных предложений и с числом слов OOV.

<br/>
Our analysis revealed that the performance of the neural machine translation suffers significantly from the length of sentences.
</br> On the Properties of Neural Machine Translation: Encoder-Decoder Approaches

Они предоставили полезный график производительности модели по мере увеличения длины предложения, который показывает потерю качества по мере усложнения предложения. 

![](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/10/Loss-in-model-skill-with-increased-sentence-length.png)

Для решения проблемы неизвестных слов, они предложили значительно увеличить словарный запас известных слов во время обучения.

Они рассматривают проблему длины предложения в сл. статье "Neural Machine Translation by Jointly Learning to Align and Translate", в которой они предложили использовать механизм внимания (attention mechanism). Вместо кодирования входной последовательности как вектора фиксированной длины, сохраняется более полная репрезентация закодированного ввода, и модель учится использовать его, чтобы обратить внимание на разные части ввода для каждого слова, выводимого декодером.

<br/>
Каждый раз, когда предлагаемая модель генерирует слово в переводе, она ищети множество позиций в исходном предложении, где сосредоточена наиболее релевантная информация. Модель затем предсказывает целевое слово на базе контекстных векторов, ассоциированных с этими исходными позициями и всеми предыдущими сгенерированными целевыми словами.
<br/>

В статье приведено очень много технических деталей, например:
* Данные подготовлены так, что в словаре хранится 30 000 наиболее распространенных слов;
* Сначала модель обучается с предложениями длиной до 20 слов а затем на предложениях длиной до 50 слов;
* Batch size = 80 предложений и модель обучается зп 4-6 эпох
* ...

На этот раз обучение модели занимает около 5 дней.

Также как и модель Sutskever'a, модель достигла результатов, которые достигали классические phrase-based SMT системы:

<br/>
Возможно более важно, что предложенный подход достиг качества перевода, сравнимой с уже существующими phrase-based SMT. Это поразительный результат, учитывая что предлагаемая архитектура и все семество систем нейронного машинного перевода предложено совсем недавно.
</br>

Cho является автором серии постов по "Introduction to Neural Machine Translation with GPUs":
* https://devblogs.nvidia.com/introduction-neural-machine-translation-with-gpus/
* https://devblogs.nvidia.com/introduction-neural-machine-translation-gpus-part-2/
* https://devblogs.nvidia.com/introduction-neural-machine-translation-gpus-part-3/


## Encoder-Decoder LSTM Networks

Encoder-Decoder LSTM - тип рекуррентной нейронной сети, спроектированной для решения задач, связанный с отображением последовательности в последовательность (seq2seq).

Спроектируем самостоятельно данную архитектуру.

### Sequence-to-Sequence Prediction Problems
Предсказание последовательностей часто включает в себя предсказание сл. значения в последовательности или вывод метки класса для входной последовательности.

Часто формируется как последовательность одного входного временного шага (input time step) в один выходной временной шаг (output time step) (т.е. one-to-one), или несколько входных временных шагов в один выходной шаг (many-to-one).

Есть более сложные задачи - на вход поступает последовательность и требуется предсказать последовательность на выходе. Эта задача называется **sequence-to-sequence prediction problem**. Проблема состоит в том, что как для входной так и выходной последовательности длина не фиксирована.

### Encoder-Decoder LSTM Architecture

Один из подходов к решению seq2seq проблемы, доказавший свою эффективность - архитектура Encoder-Decoder LSTM.

<br/>
… RNN Encoder-Decoder, consists of two recurrent neural networks (RNN) that act as an encoder and a decoder pair. The encoder maps a variable-length source sequence to a fixed-length vector, and the decoder maps the vector representation back to a variable-length target sequence.
<br/>
**Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation**

Encoder-Decoder LSTM апробирован для задач NLP, где показал выдающиеся результаты.

Новизной архитектуры являлось использование внутренней репрезентации фиксированного размера, которая кодирует входную последовательность и выходная последовательность получается из неё. По этой причине этот метод иногда называется **sequence embedding**.

Самая первая задача, на которой была апробирована архитектура была задача перевода с английского на французский язык, где внутренняя репрезентация закодированных фраз английского языка была визуализирована. Графики выявили качественную обученную структуру фраз, специально для задачи перевода.

<br/>
The proposed RNN Encoder-Decoder naturally generates a continuous-space representation of a phrase. […] From the visualization, it is clear that the RNN Encoder-Decoder captures both semantic and syntactic structures of the phrases
<br/>
**Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation**

Пр решении задачи перевода, модель оказхалась более эффективной, когда входная последовательность была перевернута. В дальнейшем, модель показала эффективность даже на очень длинных входных последовательностях.

<br/>
We were able to do well on long sentences because we reversed the order of words in the source sentence but not the target sentences in the training and test set. By doing so, we introduced many short term dependencies that made the optimization problem much simpler. … The simple trick of reversing the words in the source sentence is one of the key technical contributions of this work
<br/>
**Sequence to Sequence Learning with Neural Networks**

Этот подход также использовался вместе с входом как изображением, где используется в качестве feature extractor'a сверточная нейронная сеть, которая затем читается декодером LSTM.

<br/>
… we propose to follow this elegant recipe, replacing the encoder RNN by a deep convolution neural network (CNN). […] it is natural to use a CNN as an image encoder”, by first pre-training it for an image classification task and using the last hidden layer as an input to the RNN decoder that generates sentences
</br>
**Show and Tell: A Neural Image Caption Generator**

![](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/07/Encoder-Decoder-LSTM-Model-Architecture.png)

#### Практические применения Encoder-Decoder LSTM

* Machine Translation;
* Learning to Execute;
* Image Captioning  (e.g. generating a text description for images);
* Conversational Modeling (e.g. generating answers to textual questions);
* Movement Classification (e.g. generating a sequence of commands from a sequence of gestures);

### Реализация Encoder-Decoder LSTM в Keras

Выше уже приведен пример построения Encoder-Decoder архитектуры в Keras. В качестве иллюстрации посимвольного преобразователя.

Однако, рассмотрим общую архитектуру.

Итак, сначала входная последовательность предъявляется сети, один символ (слово, метка) в единицу времени. Нам необходим уровень кодирования (encoding level), чтобы обучиться взаимосвязям между шагами входной последовательности и разработать внутреннюю репрезентацию этих взаимоотношений.

Один или более слоев LSTM могут быть испоьзованы для реализации модели кодировшикаю Выходом этой модели будет вектор фиксированного размера, репрезентующий внутреннюю структуру входной последовательности. ЧИсло нейронов в этом ровне и определяет длину этого вектора:

In [ ]:
model = Sequential()
model.add(LSTM(..., input_shape(...)))

Декодер должен преобразовать обученную внутреннюю репрезентацию входной последовательности в коректную выходную последовательность.

Один или более LSTM слоев также могут быть использованы для реализации модели декодера. Эта модель читает вектор фиксированного размера на выходе кодировщика.

Также как и для обычной LSTM модели, необходимо использовать полносвязный слой Dense на выходе сети. Одни и те же веса могут быть использованы на выходе на каждом временном шаге (each time stepo) в выходной последовательности. Это достигается за счет wrapper'a Dense layer'a:

In [ ]:
model.add(LSTM(..., return_sequences=True))
model.add(TimeDistributed(Dense(...)))

Однако, здесь есть проблема.

Мы должны соединить кодировщик с декодером, а они не соответствуют друг другу.

То есть, кодировщик будет порождать 2 мерную матрицу результирующих значений, чья длина определяется числом нейронов на слое. 
Декодер LSTM ожидает трехмерный вход размерности [samples; time steps; features] чтобы произвести декодирование последовательности некоторой длины.

Если вы попытаетесь соединить эти шаги, то получите ошибку, с сообщением о том, что выходом декодера является 2D тензор, а входом в декодер 3D тензор.

Мы можем решить эту проблему с использованием слоя RepeatVector. Этот слой просто овторяет 2D вход, несколько раз, чтобы создать 3D етнзор.

Слой RepeatVector может быть использован как "адаптер" для соедининения частей кодировщика и декодера в сеть. Мы можем сконфигурировать RepeatVector так, чтобы повторить вектор фиксированной длины по одноу разу для каждлого временного шага в выходной последовательности.


In [ ]:
model.add(RepeatVector(...))

Собрав всё воедино получим:

In [ ]:
model = Sequential()
model.add(LSTM(..., input_shape=(...)))
model.add(RepeatVector(...))
model.add(LSTM(..., return_sequences=True))
model.add(TimeDistributed(Dense(...)))

To summarize, the RepeatVector is used as an adapter to fit the fixed-sized 2D output of the encoder to the differing length and 3D input expected by the decoder. 

The TimeDistributed wrapper allows the same output layer to be reused for each element in the output sequence.

## Attention and Memory in Deep Learning

Последние тренды в Deep Learning являются мезанизмы внимания.
Illy Sutskever в своем интервтб упомянул, что механизмы внимания самое интересное достижение в области sequence to sequence prediction.

Механизмы внимания в нейронных сетях очень и очень отдалено напоминают механизмы визуального внимания у людей. Человеческое визуальное внимание достаточно хорошо изучено, и существубт несколько разных моделей, и все они по существу сводятся к тому, чтобы сосредоточиться на определенной области изображения с "высоким разрешением", воспринимая окружение в "низком разрешении", а затем корректируя фокус с течением времени.

Механизмы внимания имеют долгую историю, особенно в распознавании изображений. Но относительно недавно механизмы внимания пришли в архитектуры рекуррентных нейронных сетей, как правило используемые в NLP.



### Какие проблемы позволяет решить механизмы внимания?

Для понимания, как может помочь нам механизм внимания, будем использовать нейронный машинный перевод (NMT). Традиционные системы машинного перевода базируются на занадочном проектировании атрибутов, основанных на статистических свойствах текста. Эти системы достаточно сложные и требуют значительных усилий инженеров и лингвистов. Системы нейронного машинного перевода работают несколько иначе. Мы отображаем значение предложения в векторную репрезентацию фиксированной длины, и затем генерируем перевод на базе этого вектора. Не полагаясь на такие вещи, как количество n-gram, а вместо этого пытаясь захватить высокоуровневую репрезентацию семантики предложения системы NMT генерализуются на новые предложения лучше, чем большинство других подходов.

Также важно то, что NMT системы можно легко построить и обучить и они не требуют какого-либо featyre ebgubeering. По факту, простая реализция в TensorFlow не больше нескольких сотен строк кода.

Большинство NMT систем работают кодируя исходное предложение (например, на немецком языке) в вектор используя рекуррентную нейронную сеть и затем декодируя английское предложение, на базе этого вектора также с использованием RNN.

![](http://www.wildml.com/wp-content/uploads/2015/09/Screen-Shot-2015-09-17-at-10.39.06-AM-1024x557.png)

На картинке выше "Echt", "Dicke", "Kiste" слова поступают в кодировщик, и после спец сигнала конца предложения (не показан здесь), декодер начинает процедуру перевода. Декодер продолжает генерацию слов до тех пор, пока не породит специальный токен конца предложения. Здесь векторы $h$ репрезентуют внутреннее состояние декодера.

Если рассмотреть внимательнее, можно увидеть что декодер должен генерировать перевод исключительно на базе **последнего** скрытого состояния $h_3$ кодировщика.
Этот вектор $h_3$ должен закодировать всё что он знает о целевом предложении. Он должен полностью захватывать семантиру этого предложения. В более технических терминах, этот вектор является sentence embedding. 

Те не менее, кажется несколько необоснованным предполагать, что мы можем кодировать всю информацию о потенциально очень длинном предложении в один вектор, чтобы декодер затем сделал точный перевод, основанный на таком "сжатом" векторе.

Допустим, исходное предложение имеет длину в 50 слов. Первое слово английского перевода вероятно сильно коррелирует с первым словом исходного предложенияю Но это означает, что декодер должен рассматривать информацию, которая была 50 шагов назад! RNN имеет проблемы с long-range dependencies. В теории, LSTM архитектуры способны иметь дело с ними, но на практике long-range dependencies продолжают оставаться очень проблематичными. Например, исследователи обнаружили, что "переворачивание" исходной последовательности дает значительно лучшие результаты, поскольку образует кратчайший путь от декодера к релевантным частям кодировщика. 

Или, например, подавая входную последовательность дважды (Learning to Execute, https://arxiv.org/abs/1410.4615), также может помочь сети лучше запомнить многие вещи.

Но всё же, этот подход с переворачиванием предложения рассматривается как т.н. workaround. Он делает вещи лучше на практике, но это не может быть принципиальным решением. Большинство переводов в соревнованиях используют такие языки как французский, или немецкий, которые достаточно похожи на английский (даже порядок слов китайского языка очень похож на английский).

Но другие языки, например японский, где последнее слово в предложении может сильно коррелировать с первым словом английского перевода? В этом случае, переворачивание входной последовательности сделает производительность намного хуже.

Решением могут быть механизмы внимания.

С использованием механизмов внимания мы больше не пытаемся закодировать исходное предложение целиков в один вектор фиксированной длины. Вместо этого, мы позволяем декодеру "обращать внимание" на разные части исходного предложения на каждом шаге генерации перевода. Важно то, что мы позволяем модели **учиться** на что обратить внимание в исходном предложении и как это коррелирует с тем, что до сих пор уже переведено. Итак, в языках, которые уже достаточно хорошо выравнены (английский и немецкий), декодер, возможно, будет обращать внимание на последовательные слова. Внимание на первое слово, когда переводм первое английское слово и т.д.

Это то, что делается в работе *Neural Machine Translation by Jointly Learning to Align and Translate* и выглядит сл. образом:
![](http://www.wildml.com/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.16.08-PM-235x300.png)

Здесь, $y$'и - наши переводы слов, порожденные декодером, а $x$'ы наши исходные слова.

Иллюстрация использует бинаправленную рекуррентную сеть, но это не важно для понимания, и сейчас можно просто игнорировать обратное направление. Важной частью является то, что каждое декодируемое выходное слово $y_t$ сейчас зависит от **взвешенной комбинации всех входных состояний**, а не только от последнего.

$a$ - весовые коэффициенты, определяющие как сильно каждое входное состояние должно рассматриваться для каждого выхода. Напроимер, $a_{3,2}$ - большое число, и это означает, что декодер обратит б**о**льшее внимание на второе состояние в исходном предложениии во время построения третьего слова целевого предложения. Значения $a$ как правило нормализуются, чтобы соответствовать законам вероятностного распределения.

Большим преимуществом механизма внимания является способность интерпретации и визуализации того, что модель делает. Например, визуализируя attention weight matrix $a$, при переводе преложения, мы можем понять **как** модель переводит:

![](http://www.wildml.com/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.23.48-PM-279x300.png)

Здесь можно увидеть перевод с французского на английский, сеть обращает внимание последовательно на каждое входное состояние, но иногда она обращает внимание на два слова, во время порождения выхода.

Например, при переводе "la Syrie" в "Syria".

### Стоимость внимания

Если взглянуть на уравнение для внимания, можно увидеть что внимание "стоит" дорого в плане вычислительных ресурсов. Нам нужно вычислить значение внимания для каждой комбинации входного и выходных слов. Если у нас есть входная последовательность из 50 слов, и генерация выходной последовательности из 50 слов, у нас будет 2500 attention values. Это не так плохо, но если вы рассматриваете character-based вычисления и имеете дело с последовательностями, состоящими из сотен докенов, механизмы внимания могут стать чрезвычайно дорогостоящими.

По факту, это достаточно контринтуитивно. Человеческое внимание - это что-то что помогает нам сэкономит вычислительные ресурсы. Фокусируясь на одних вещах, мы можем пренебрегать многими другими вещами. Но это не совсем то, что мы делаем в вышеупомянутой модели. Мы должны рассмотреть всё подробно, и лишь потом решить на чём сфокусироваться. Интуитивно, это эквивалентно выводу переведенного слова, а затем "шаг назад" через всю внутреннюю память текста, чтобы решить какое слово должно быть произведено следующим. Это выглядит избыточным, и явно не то, что делают люди. По факту это больше напоминает доступ к памяти, а не внимание. Тем не менее, это не стало помехой тому, чтобы стать довольно популярным и хорошо решать многие задачи.

Альтернативным подходом к вниманию является использование Reinforcement Learning для предсказания примерного положения на котором следует сфокусироваться.

### Внимание не в машинном переводе

До сих пор рассматривалось внимание, применительно к машинному переводу. Но тот же самый механизм может быть применен к любой рекуррентной модели.

В работе *Show, Attent and Tell* авторы применили механизмы внимания к проблеме генерации описаний изображений. Они использовали сверточную нейронную сеть для кодирования изображения и рекуррентную нейронную сеть с механизма ми внимания для генерации описания. Визуализация весов внимания (attention weight), такая же как и в примере с машинным переводом и позволяет нам интерпретировать на что обращает внимание модель при генерации слов.

![](http://www.wildml.com/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.42.58-PM-768x572.png)

В работе *Grammar as a Foreign Language* авторы использовали рекуррентные нейронные сети с механизмом внимания для генерации деревьев синтаксического разбора. Визуализация матрицы внимания дает понимание, как эта сеть генерирует деревья синтаксического разбора:

![](http://www.wildml.com/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.49.19-PM-768x714.png)

В работе *Teaching Machines to Read and Comprehend*, авторы использовали RNN для чтения текста, чтения вопроса и порождения ответа на этот вопрос. Визуализация матрицы внимания позволяет увидеть, куда сеть "смотрит", когда пытается найти ответ на вопрос:
![](http://www.wildml.com/wp-content/uploads/2015/12/Screen-Shot-2015-12-30-at-1.51.19-PM-768x336.png)



### Attention = (Fuzzy) Memory?

Основная проблема, которую решает механизм внимания, заключается в том, что она позволяет сети возвращаться к входной последовательности, вместо принуждения её кодировать всю информацию в один вектор фиксированной длины.
Возможно, это некорректное название. Мы можем интерпретировать механизм внимания по другому: механизм внимания просто передает сети дорступ к внутренней памяти, которая является скрытым состоянием кодировщика (encoder'a). В этой интепретации, вместо выбора "на что обратить внимание", нейронная сеть выбирает, что извлечь из памятию В отличии от типичной памяти, доступ к механизму памяти щлест "мягкий", что означает что сеть извлекает взвешенную комбинацию всех локаций памяти, а не значения из одной дискретной локации. 

Позволяя мягкий доступ к памяти (soft memory access) мы получаем преимущество легкого обучения сети с использованием обратного распросстранения ошибки (хотя есть non-fuzzy подходы, где градиенты вычисляются с использованием методов сэмплирования, а не обратного распространения ошибки).

Механизмы памяти сами по себе имеют очень длинную историю. Скрытое состояния обычной RNN само по себе является типом внутренней памяти. Но RNN страдает от проблемы "затухающего градиента" (vanishing gradient problem), что не позволяет им изучать долгосрочные зависимости (long-range dependencies). LSTM сети улучшили состояние дел, добавив механизм гейтов, позволяющий явно удалять и обновлять память.

Продолжается тенденция усложнения структур памяти. *End-to-end Memory networks* позволяют сети считывать одну и ту же последовательность несколько раз, прежде чем будет сгенерирован вывод, обновляя содержимое памяти на каждом шаге. Например, отвечая на вопрос, делается несколько "шагов рассуждений" над исходной историей. Однако, когда параметры весов сети связаны некоторым образом, механизмы памяти в End-to-End Memory Networks идентичны механизмам внимания.

Источник: http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/


## Google Neural Machine Translation Systems

* https://research.googleblog.com/2016/09/a-neural-network-for-machine.html
* https://research.googleblog.com/2016/11/zero-shot-translation-with-googles.html

## Инструментарий

Каждый раз самостоятельно разрабатывать архитектуру кодировщика и декодера может быть чрезвычайно утомительно.
Однако, стандартная бибилотека Keras не имеет никаких высокоуровневых "строительных блоков" для построения моделей Sequence-to-sequence.

Однако, имеются дополнительные библиотеки: 
Seq2Seq: https://github.com/farizrahman4u/seq2seq

Эта библиотека имеет зависимость RecurrentShop: https://www.github.com/farizrahman4u/recurrentshop библиотека, облегчающая построение рекуррентных нейронных сетей.

Вернемся к библиотеке seq2seq.

![](https://camo.githubusercontent.com/242210d7d0151cae91107ee63bff364a860db5dd/687474703a2f2f6936342e74696e797069632e636f6d2f333031333674652e706e67) 

Библиотека содержит модульные слои, которые можно использовать при построении собственных seq2seq моделей из коробки.
Являясь по своей сути строительными блоками, они могут использоваться как слои в больших системах. 

Каждая модель имеет два базовых подслоя:
* кодировщик (encoder)
* декодер (decoder).

Кодировщик производит преобразование входной последовательности во внутреннюю репрезентацию, называемую контекстным вектором, которая используется для декодирования и генерации выходной последовательности.

Длина входной и выходной последовательностей может отличаться друг от друга, и не существует явного отношение между входными и выходными последовательностями. В дополнение к слоям кодировщика и декодера, модель Seq2Seq может также содержать слои, такие как left-stack (Stacked LSTM на стороне кодировщика), right-stack (Stacked LSTM на стороне декодера), resizers (для обеспечения сопоставления размерности между кодировщиком и декодером), и слои dropout'a.

Рассмотрим код простой модели:

In [ ]:
import seq2seq
from seq2seq.models import SimpleSeq2Seq

model = SimpleSeq2Seq(input_dim=5, hidden_dim=10, output_length=8, output_dim=8)
model.compile(loss='mse', optimizer='rmsprop')

Глубокая Seq2Seq модель:

In [ ]:
import seq2seq
from seq2seq.models import SimpleSeq2Seq

model = SimpleSeq2Seq(input_dim=5, hidden_dim=10, output_length=8, output_dim=8, depth=3)
model.compile(loss='mse', optimizer='rmsprop')

Заметим, что мы указали глубину как для кодировщика, так и для декодера равную 3 и Ваша модель имеет общую глубину, равную 6.

Вы можете явно указать различную глубин как для кодировщика, так и для декодера:

In [ ]:
import seq2seq
from seq2seq.models import SimpleSeq2Seq

model = SimpleSeq2Seq(input_dim=5, hidden_dim=10, output_length=8, output_dim=20, depth=(4, 5))
model.compile(loss='mse', optimizer='rmsprop')

#### Продвинутые Seq2Seq модели

До сих пор использовалась модель SimpleSeq2Seq, которая является самой минимиластичной.

В фактической реализации Seq2Seq, описанной в [1] скрытое состояние кодировщика передается в декодер. Также, выход декодера на кажом временном шаге становится входом декодера на сл. временном шаге.



In [ ]:
import seq2seq
from seq2seq.models import Seq2Seq

model = Seq2Seq(batch_input_shape=(16, 7, 5), hidden_dim=10, output_length=8, output_dim=20, depth=4)
model.compile(loss='mse', optimizer='rmsprop')

Заметим, что нужно явно специфицировать полную размерность входа, включая число объектов.
Это обсловлено необходимостью статического скрытого состояния для передачи его через уровни.

#### Peeky Seq2Seq model

В библиотеке есть и реализация модели Cho, где декодер может заглядывать в контекстный вектор на каждом шаге.
Для этого просто указывается параметр 

    peek=True:

In [ ]:
import seq2seq
from seq2seq.models import Seq2Seq

model = Seq2Seq(batch_input_shape=(16, 7, 5), hidden_dim=10, output_length=8, output_dim=20, depth=4, peek=True)
model.compile(loss='mse', optimizer='rmsprop')

Attention Model также доступна:
Все модели, описанные ранее не имеют какой-либо концепции выравнивания между элементами входной последовательности и последовательностью выходных элементов. Для машинного перевода, обучение выравниванию между входными и выходными последовательностями значительно улучшает производительность. 
![](https://camo.githubusercontent.com/0e2e4e5fb2dd47846c2fe027737a5df5e711df1b/687474703a2f2f6936342e74696e797069632e636f6d2f6132727733642e706e67)


Инструментарий Seq2seq включает готовую к использувонию модель с вниманием, которая как раз и содержит в себе выравнивание. Заметим, что в attention модели нет распространения скрытого состояния, и бинаправленная LSTM используется по умолчанию.

In [ ]:
import seq2seq
from seq2seq.models import AttentionSeq2Seq

model = AttentionSeq2Seq(input_dim=5, input_length=7, hidden_dim=10, output_length=8, output_dim=20, depth=4)
model.compile(loss='mse', optimizer='rmsprop')

**Дополнительные ссылки:**

* Gentle Introduction to Global Attention for Encoder-Decoder Recurrent Neural Networks https://machinelearningmastery.com/global-attention-for-encoder-decoder-recurrent-neural-networks/

* How to Develop an Encoder-Decoder Model with Attention for Sequence-to-Sequence Prediction in Keras: https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/

* Implementation Patterns for the Encoder-Decoder RNN Architecture with Attention: https://machinelearningmastery.com/implementation-patterns-encoder-decoder-rnn-architecture-attention/

##### Статьи по теме
* "Sequence to Sequence Learning with Neural Networks" ( https://arxiv.org/pdf/1409.3215.pdf )
* "Learning Phrase Representations using RNN Encoder–Decoder for Statistical Machine Translation" ( https://arxiv.org/pdf/1406.1078.pdf )
* "Massive Exploration of Neural Machine Translation Architectures" ( https://arxiv.org/pdf/1703.03906.pdf )
* "Google’s Neural Machine Translation System: Bridging the Gap between Human and Machine Translation" ( https://arxiv.org/pdf/1609.08144.pdf )
* "On the Properties of Neural Machine Translation: Encoder–Decoder Approaches" ( https://arxiv.org/pdf/1409.1259.pdf )
* "Attention Is All You Need" ( https://arxiv.org/pdf/1706.03762.pdf )
* "Neural Machine Translation by Jointly Learning to Align and Translate" ( https://arxiv.org/pdf/1409.0473 )

##### Посты в блогах
* https://medium.com/@devnag/seq2seq-the-clown-car-of-deep-learning-f88e1204dac3
* "Sequence Modeling With Neural Networks (Part 1): Language & Seq2Seq" https://indico.io/blog/sequence-modeling-neuralnets-part1/
* "Sequence Modeling with Neural Networks (Part 2): Attention Models" https://indico.io/blog/sequence-modeling-neural-networks-part2-attention-models/
* "Sequence to sequence model: Introduction and concepts" https://towardsdatascience.com/sequence-to-sequence-model-introduction-and-concepts-44d9b41cd42d
* "A ten-minute introduction to sequence-to-sequence learning in Keras" https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
* "Attention and Memory in deep learning and NLP" http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/
* https://datascience.stackexchange.com/questions/14581/when-to-use-gru-over-lstm
* https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/

##### Лабораторная работа
* Может быть как то абсолютно случайно сможет помочь статья: https://machinelearningmastery.com/develop-neural-machine-translation-system-keras/

Introduction to Neural Machine Translation with GPUs:
* https://devblogs.nvidia.com/introduction-neural-machine-translation-with-gpus/
* https://devblogs.nvidia.com/introduction-neural-machine-translation-gpus-part-2/
* https://devblogs.nvidia.com/introduction-neural-machine-translation-gpus-part-3/


##### Библиотеки
* https://github.com/farizrahman4u/seq2seq





